In [1]:
# univariate multi-step encoder-decoder cnn-lstm for the power usage dataset
from math import sqrt
from numpy import split
from numpy import array
from numpy import mean
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pdb
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import pickle

Using Theano backend.


In [2]:
def split_dataset(n_input):
    data = SCALED_DATA
    for i in range(STEPS):
        data = np.delete(data, -1, 0)
    
    number_of_rows = len(data)
    train_number = (number_of_rows * TRAIN_PERCENTAGE) / 100
    test_number = (number_of_rows * TEST_PERCENTAGE) / 100


    while True:
        if train_number % n_input == 0 and test_number % n_input == 0:
            break
        else:
            number_of_rows = number_of_rows - 1
            train_number = (number_of_rows * TRAIN_PERCENTAGE) / 100
            test_number = (number_of_rows * TEST_PERCENTAGE) / 100

    print(train_number)
    print(test_number)
    total_valid_count = int(train_number + test_number)
    print(total_valid_count)
    rows_to_remove = len(data) - total_valid_count
    print(rows_to_remove)

    data = data[rows_to_remove:]
    number_of_rows = len(data)
    train_number = int((number_of_rows * TRAIN_PERCENTAGE) / 100)
    test_number = int((number_of_rows * TEST_PERCENTAGE) / 100)
    
    train = data[:train_number]
    test = data[train_number:]
    print(f'Train length {len(train)}')
    print(f'Test length {len(test)}')

    train = array(split(train, len(train)/n_input))
    test = array(split(test, len(test)/n_input))

    return train, test

In [3]:
def build_model(train, config):
    n_input, epochs, batch_size, conv1, conv2 = config[0], config[1], config[2], config[3], config[4]
    lstm, dense, conv_layer = config[5], config[6], config[7]
    
    train_x, train_y = to_supervised(train, n_input, STEPS)
    verbose = 0
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1] 

    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

    model = Sequential()
    model.add(Conv1D(conv1, conv_layer, activation='relu', input_shape=(n_timesteps,n_features))) 
    model.add(Conv1D(conv2, conv_layer, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(RepeatVector(n_outputs))
    model.add(LSTM(lstm, activation='relu', return_sequences=True)) 
    model.add(TimeDistributed(Dense(dense, activation='relu'))) 
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='adam')
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    if SAVE_MODEL:
        pickle.dump(model, open(MODEL_NAME, 'wb'))
        pickle.dump(PRICE_SCALER, open('price_scaler.sav', 'wb'))
        pickle.dump(SCALER, open('scaler.sav', 'wb'))
    return model

In [4]:
# evaluate a single model
def evaluate_model(train, test, config):
    dataset = SCALED_DATA
    n_input = config[0]
    
    model = build_model(train, config)
    history = [x for x in train]
    predictions = list()
    actual = []

    unscaled_test_prices = PRICE_SCALER.inverse_transform(test[:, :, 0])
    unscaled_test_prices = unscaled_test_prices.reshape(len(test), test.shape[1], 1)
    
    for i in range(len(test)):
        history.append(test[i, :])
        if len(test) > 1 and i > 0:
            actual = np.append(actual, unscaled_test_prices[i][:STEPS,0])

        yhat_sequence = forecast(model, history, n_input)
        predictions.append(yhat_sequence)
        
    predictions = array(predictions).reshape(len(predictions), STEPS)
    predictions = PRICE_SCALER.inverse_transform(predictions)
    
    last_actual_price = PRICE_SCALER.inverse_transform(dataset[-STEPS:,0].reshape(1,STEPS))
    actual = np.append(actual, last_actual_price)
    actual = actual.reshape(len(predictions), STEPS)
    
    score, scores = evaluate_forecasts(actual, predictions)
    return score, actual, predictions

In [5]:
def evaluate_forecasts(actual, predicted):
    scores = list()
    for i in range(STEPS):
        mse = mean_squared_error(actual[:, i], predicted[:, i])
     #   print(f'TEST actual price {actual[:, i]}!')
     #   print(f'TEST predicted price {predicted[:, i]}!')
        rmse = sqrt(mse)
        scores.append(rmse)
    s=0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2

    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

In [6]:
def forecast(model, history, n_input):
    data = array(history)
    data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))

    input_x = data[-n_input:, :]
    input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
    #pdb.set_trace()
    yhat = model.predict(input_x, verbose=0)
    yhat = yhat.reshape(1,STEPS)

    return yhat

In [7]:
def to_supervised(train, n_input, n_out):
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0

    for _ in range(len(data)):
        in_end = in_start + n_input
        out_end = in_end + n_out
        
        if out_end <= len(data):
            X.append(data[in_start:in_end, :])
            y.append(data[in_end:out_end, 0])
            
        in_start += 1
    return array(X), array(y)

In [8]:
def summarize_scores(name, score, scores):
    s_scores = ', '.join(['%.1f' % s for s in scores]) 
    print('%s: [%.3f] %s' % (name, score, s_scores))

In [9]:
def model_configs():
    # define scope of configs
    n_input = [10]
    n_epochs = [95]
    n_batch = [450] 
    conv1 = [220]
    conv2 = [180]
    lstm = [100]
    dense = [180]
    conv_layer = [4]
    # create configs
    configs = list()
    for i in n_input:
        for j in n_epochs:
            for k in n_batch:
                for l in conv1:
                    for m in conv2:
                        for n in lstm:
                            for p in dense:
                                for q in conv_layer:
                                    cfg = [i, j, k, l, m, n, p, q]
                                    configs.append(cfg)
    print('Total configs: %d' % len(configs)) 
    return configs

In [10]:
def repeat_evaluate(train, test, config, times=30):
    scores = 0
    all_actuals, all_predictions = [], []
    for _ in range(times):
        score, actual, predictions = evaluate_model(train, test, config)
        all_actuals.append(actual)
        all_predictions.append(predictions)
        scores += score
    
    average_score = scores / times
    print(f'Average score {average_score}')
    
    print(f'Last actual prices {all_actuals[-1][-8:]}')
    print(f'Last predicted prices {all_predictions[-1][-8:]}')
    return average_score

In [11]:
TRAIN_PERCENTAGE = 75
TEST_PERCENTAGE = 25
STEPS = 1
REPEAT = 1
SAVE_MODEL = False
MODEL_NAME = 'LSTM_1_step.sav'

SCALER = StandardScaler()
PRICE_SCALER = StandardScaler()

dataset = read_csv('dax_5_train.csv', header=0, infer_datetime_format=False, parse_dates=['price_date'], index_col=['price_date'])
dataset = dataset.round(2)
normal_values = dataset.values[:, 1:]
price_values = dataset.values[:, 0]
price_values = price_values.reshape(len(price_values), 1)

PRICE_SCALER = PRICE_SCALER.fit(price_values)
SCALER = SCALER.fit(normal_values)

SCALED_PRICES = PRICE_SCALER.transform(price_values)
SCALED_NORMAL = SCALER.transform(normal_values)

SCALED_DATA = np.column_stack((SCALED_PRICES, SCALED_NORMAL))

configs = model_configs()
np.set_printoptions(suppress=True)

scores = []
print(f'CONFIGS {len(configs)}')
i = 0
for config in configs:
    i += 1
    print(f'CONFIG NUMBER {i}')
    n_input = config[0]
    train, test = split_dataset(n_input)
    score = repeat_evaluate(train, test, config, REPEAT)
    config.insert(0, score)
    scores.append(config)

scores.sort(key = lambda scores: scores[0])
print(f'Best score {scores[0]}')
print(f'All scores {scores}')


#score, scores, model = evaluate_model(train, test, config)

#repeat_evaluate(data, config, n_test)

#summarize_scores('lstm', score, scores)
#pyplot.plot(scores, marker='o', label='lstm') 
#pyplot.show()

Total configs: 1
CONFIGS 1
CONFIG NUMBER 1
6540.0
2180.0
8720
27
Train length 6540
Test length 2180
Average score 10.821406637753379
Last actual prices [[12582. ]
 [12664. ]
 [12629.8]
 [12635.5]
 [12641.5]
 [12652.3]
 [12643.5]
 [12634.9]]
Last predicted prices [[12571.673]
 [12656.981]
 [12626.987]
 [12627.821]
 [12646.622]
 [12651.83 ]
 [12650.816]
 [12635.155]]
Best score [10.821406637753379, 10, 95, 450, 220, 180, 100, 180, 4]
All scores [[10.821406637753379, 10, 95, 450, 220, 180, 100, 180, 4]]


In [12]:
test_value = test[0].reshape(STEPS,10,21)
print(test[1][0][1])

-0.9919784062012623


In [13]:

#print(scaler.transform(test[1]))

#print(encode_test)

In [14]:
#print(scaler.inverse_transform(encode_test))


In [15]:
MODEL_NAME = 'LSTM_1_step.sav'
loaded_model = pickle.load(open(MODEL_NAME, 'rb'))
predicted_price = loaded_model.predict(test_value)
print(PRICE_SCALER.inverse_transform(test[1][0][1].reshape(1,1)))
PRICE_SCALER.inverse_transform(predicted_price)

[[12057.37021315]]


array([[[11968.779]]], dtype=float32)

In [16]:
price_scaler = pickle.load(open('price_scaler.sav', 'rb'))
to_predict = test[-1].reshape(1, 10,21)
#print(price_scaler.inverse_transform(to_predict))
prediction = loaded_model.predict(to_predict)
print(PRICE_SCALER.inverse_transform(prediction))
print(price_scaler.inverse_transform(prediction))
#print(price_scaler.inverse_transform(test[-1]))
print(test[-1])


[[[12656.607]]]
[[[12626.908]]]
[[ 2.17015422  1.44500499 -1.35848781  1.33695231 -1.29027577 -1.03391388
  -0.97457853 -0.40739564  0.18208831  2.19836513  2.18472879  2.17237978
   2.17437232 -0.50370096 -0.70185368  2.172128   -0.49124932 -0.647236
  -1.10601423 -1.86362254 -0.49708764]
 [ 2.18256258  1.17422906 -0.53940645  0.82447931 -1.29027577 -0.75729327
  -0.42330218 -0.354462    0.41525944  2.19658161  2.18494477  2.17129883
   2.18352463 -0.39451565 -0.15421556  2.16943047 -0.36424864 -0.12015234
  -0.51556234 -0.58035213 -0.60792167]
 [ 2.20414232  0.90345312 -0.81243357  0.31200631  1.27304631 -0.40306522
   0.34007759 -0.50363863 -0.17797428  2.19906773  2.19228814  2.18859404
   2.19321532 -0.15014852  0.20607268  2.18399713 -0.17374762  0.23123677
   0.28328435  0.63078333 -0.53134544]
 [ 2.18795751  0.63267719 -1.08546069 -0.20046669  1.27304631 -0.86678893
  -0.5317605  -0.66243956 -0.41307777  2.19814894  2.19180218  2.1983226
   2.18513975 -0.39451565  0.06195739  2

In [17]:
([[12450. ],
       [12433.2],
       [12438.5],
       [12439.7],
       [12444.7],
       [12435.7],
       [12439.8],
       [12466.5],
       [12465.5],
       [12465.5],
       [12430.2],
       [12385.5]])

[[12450.0],
 [12433.2],
 [12438.5],
 [12439.7],
 [12444.7],
 [12435.7],
 [12439.8],
 [12466.5],
 [12465.5],
 [12465.5],
 [12430.2],
 [12385.5]]